In [1]:
class profile:
    sample_profile={"advisors":[{"ids":[{"value":"INSPIRE-00133260",
                                    "schema":"INSPIRE ID"}],
                "name":"Valle, Jose W.F.",
                "hidden":False,
                "record":{"$ref":"https://inspirehep.net/api/authors/985058"},
                "degree_type":"phd","curated_relation":False},
               {"name":"Ponce, William A.","hidden":False,
                "degree_type":"master",
                "curated_relation":False}],
                "email_addresses":[{"value":"restrepo@udea.edu.co","current":True}],
    "positions":[{"rank":"SENIOR",
                  "hidden":False,
                  "record":{"$ref":"https://inspirehep.net/api/institutions/903906"},
                  "current":True,
                  "start_date":"2004",
                  "institution":"Antioquia U.",
                  "curated_relation":True},
                 {"rank":"PHD",
                  "hidden":False,
                  "record":{"$ref":"https://inspirehep.net/api/institutions/907907"},
                  "current":False,
                  "end_date":"2001","start_date":"1997",
                  "institution":"Valencia U., IFIC","curated_relation":True}],
    "ids":[{"value":"D.Restrepo.1","schema":"INSPIRE BAI"},
           {"value":"INSPIRE-00119748","schema":"INSPIRE ID"},
           {"value":"0000-0001-6455-5564","schema":"ORCID"},
           {"value":"diego-restrepo-209b7927","schema":"LINKEDIN"}],
    "name":{"value":"Restrepo Quintero, Diego Alejandro",
            "preferred_name":"Diego Restrepo"},
    "stub":False,
    "urls":[{"value":"http://gfif.udea.edu.co"},
            {"value":"https://scholar.google.com/citations?user=1sKULCoAAAAJ"},
            {"value":"https://www.researchgate.net/profile/Diego_Restrepo2"}],
    "status":"active",
    "$schema":"https://inspirehep.net/schemas/records/authors.json",
    "deleted":False,
    "control_number":991924,
    "legacy_version":"20210323213044.0",
    "arxiv_categories":["hep-ph"],
    "legacy_creation_date":"1999-08-23"}
    def __init__(self,*args):
        if args:
            Self=args[0]
            if Self.is_profile:
                self.is_profile=Self.is_profile
                self.name=Self.name
                self.positions=Self.positions
                self.email_addresses=Self.email_addresses
                self.ids=Self.ids
        else:
            self.is_profile=False
            self.name=''
            self.positions=[]
            self.email_addresses=[]
            self.ids=[]
            
            
    def author(self,p):
        '''
        Select:
        * current position
        * non-INSPIRE ids
        * name
        * control_number
        * arxiv_categories
        '''
        self.is_profile=True
        self.name=p.get('name')
        self.positions=p.get('positions')
        self.email_addresses=p.get('email_addresses')
        self.ids=p.get('ids')
        
        return {'name':self.name,
                'positions':self.positions,
                'email_addresses':self.email_addresses,
                'ids':self.ids}
    
        
        

In [21]:
au=profile()
p=au.sample_profile
assert au.author(p)['name']['value']=='Restrepo Quintero, Diego Alejandro'
assert profile(au).name.get('value')=='Restrepo Quintero, Diego Alejandro'

In [51]:
str({'ids': [{'value': 'D.Restrepo.1', 'schema': 'INSPIRE BAI'}],
                   'uuid': '300c9b2c-15b0-4937-ad36-8ccff33fd09d',
    'emails': ['restrepo@udea.edu.co'],
    'record': {'$ref': 'https://inspirehep.net/api/authors/991924'},
    'full_name': 'Restrepo, Diego',
    'affiliations': [{'value': 'Antioquia U.',
      'record': {'$ref': 'https://inspirehep.net/api/institutions/903906'}},
     {'value': 'Campinas State U.',
      'record': {'$ref': 'https://inspirehep.net/api/institutions/902714'}}],
    'signature_block': 'RASTRAPd',
    'raw_affiliations': [{'value': 'Instituto de Física, Universidad de Antioquia, Calle 70 No 52-21, Medellín, Colombia'},
     {'value': 'Instituto de Física Gleb Wataghin, UNICAMP, 13083-859, Campinas, SP, Brazil'}]} ).replace("'",'"')

'{"ids": [{"value": "D.Restrepo.1", "schema": "INSPIRE BAI"}], "uuid": "300c9b2c-15b0-4937-ad36-8ccff33fd09d", "emails": ["restrepo@udea.edu.co"], "record": {"$ref": "https://inspirehep.net/api/authors/991924"}, "full_name": "Restrepo, Diego", "affiliations": [{"value": "Antioquia U.", "record": {"$ref": "https://inspirehep.net/api/institutions/903906"}}, {"value": "Campinas State U.", "record": {"$ref": "https://inspirehep.net/api/institutions/902714"}}], "signature_block": "RASTRAPd", "raw_affiliations": [{"value": "Instituto de Física, Universidad de Antioquia, Calle 70 No 52-21, Medellín, Colombia"}, {"value": "Instituto de Física Gleb Wataghin, UNICAMP, 13083-859, Campinas, SP, Brazil"}]}'

{'ids': [{'value': 'D.Restrepo.1', 'schema': 'INSPIRE BAI'}],
 'uuid': '300c9b2c-15b0-4937-ad36-8ccff33fd09d',
 'emails': ['restrepo@udea.edu.co'],
 'record': {'$ref': 'https://inspirehep.net/api/authors/991924'},
 'full_name': 'Restrepo, Diego',
 'affiliations': [{'value': 'Antioquia U.',
   'record': {'$ref': 'https://inspirehep.net/api/institutions/903906'}},
  {'value': 'Campinas State U.',
   'record': {'$ref': 'https://inspirehep.net/api/institutions/902714'}}],
 'signature_block': 'RASTRAPd',
 'raw_affiliations': [{'value': 'Instituto de Física, Universidad de Antioquia, Calle 70 No 52-21, Medellín, Colombia'},
  {'value': 'Instituto de Física Gleb Wataghin, UNICAMP, 13083-859, Campinas, SP, Brazil'}]}

In [84]:
import requests
import time
class empty_json:
    status_code=0
    def json(self):
        return {}
    
class author(profile):
    sleep=0.4
    sample_author={"ids": [{"value": "D.Restrepo.1", "schema": "INSPIRE BAI"}], 
                   "uuid": "300c9b2c-15b0-4937-ad36-8ccff33fd09d", 
                   "emails": ["restrepo@udea.edu.co"], 
                   "record": {"$ref": "https://inspirehep.net/api/authors/991924"}, 
                   "full_name": "Restrepo, Diego", 
                   "affiliations": [{"value": "Antioquia U.", 
                                     "record": {"$ref": "https://inspirehep.net/api/institutions/903906"}}], 
                   "signature_block": "RASTRAPd", 
                   "raw_affiliations": [{"value": "Instituto de Física, Universidad de Antioquia, Calle 70 No 52-21, Medellín, Colombia"}, 
                                        {"value": "Instituto de Física Gleb Wataghin, UNICAMP, 13083-859, Campinas, SP, Brazil"}]}
    def __init__(self,*args,**kwargs):
        super(author, self).__init__(*args,**kwargs)
        if args:
            Self=args[0]
            if Self.is_author:
                self.is_author=Self.is_author
                self.full_name=Self.full_name
                self.author_id=Self.author_id
        else:
            self.is_author=False
            self.full_name=''
            self.author_id=''
        
    def author(self,a):
        #TODO: Check previous analysis for more metadata
        #use requests
        self.is_author=True
        
        r=empty_json()
        profile={}
        self.author_id=''
        if isinstance(a.get('record'),dict):
            url=a.get('record').get("$ref")
            if url and isinstance(url,str):
                self.author_id=url.split('/')[-1]
                r=requests.get(url)
                time.sleep(self.sleep)
        if r.status_code==200:
            profile=r.json().get('metadata')
        d=super(author, self).author(profile)
        self.full_name=a.get('full_name')
        d['full_name']=self.full_name
        d['author_id']=self.author_id
        return d

In [101]:
au=author()
a=au.sample_author
assert au.author(a).get('author_id')=='991924' #Check output
assert au.author_id=='991924' #Check object
nau=author(au)
assert au.author_id=='991924' #Check initialization

In [ ]:
au.author_id

In [95]:
au.is_author

True

In [96]:
nau=author(au)

In [99]:
nau.author_id

'991924'

In [91]:
au.author_id

''

In [57]:
au.email_addresses

[{'value': 'restrepo@udea.edu.co', 'current': True}]

In [44]:
au.name

{'value': 'Restrepo Quintero, Diego Alejandro',
 'preferred_name': 'Diego Restrepo'}

In [6]:
#in general len(l)!=0
authors=inspire.literature(l)

authors → list of authors object
```
[a_1,a_2,...a_n]
```
where `a_i` is just a dictionary which is either created or updated

In [ ]:
aus=inspire.legacy_instituion(legacy_instituion)
aus=inspire.country(country)
aus=inspire.authors(inspire_authors)
au=inspire.author(inspire_author)

In [ ]:
au =inspire.profile(profile) # → a_i

Input:

In [ ]:
authors=inspire.read_json('authors.json')
authors=inspire.read_list(author_list)

In [45]:
import requests

In [46]:
r=requests.get('https://inspirehep.net/api/literature/2080612')

In [47]:
r.json()

{'id': '2080612',
 'uuid': '24e839a4-3520-48a2-9526-593e2f00684e',
 'metadata': {'citation_count_without_self_citations': 0,
  'citation_count': 0,
  'core': True,
  'titles': [{'title': 'Dirac dark matter, neutrino masses, and dark baryogenesis',
    'source': 'arXiv'}],
  '$schema': 'https://inspirehep.net/schemas/records/hep.json',
  'authors': [{'ids': [{'value': 'D.Restrepo.1', 'schema': 'INSPIRE BAI'}],
    'uuid': '300c9b2c-15b0-4937-ad36-8ccff33fd09d',
    'emails': ['restrepo@udea.edu.co'],
    'record': {'$ref': 'https://inspirehep.net/api/authors/991924'},
    'full_name': 'Restrepo, Diego',
    'affiliations': [{'value': 'Antioquia U.',
      'record': {'$ref': 'https://inspirehep.net/api/institutions/903906'}},
     {'value': 'Campinas State U.',
      'record': {'$ref': 'https://inspirehep.net/api/institutions/902714'}}],
    'signature_block': 'RASTRAPd',
    'raw_affiliations': [{'value': 'Instituto de Física, Universidad de Antioquia, Calle 70 No 52-21, Medellín, Colom

In [4]:
Can be exported to JSON or Excel

{'id': '991924',
 'uuid': 'c233167c-7187-4235-979d-0f5bb68aab97',
 'metadata': {'advisors': [{'ids': [{'value': 'INSPIRE-00133260',
      'schema': 'INSPIRE ID'}],
    'name': 'Valle, Jose W.F.',
    'hidden': False,
    'record': {'$ref': 'https://inspirehep.net/api/authors/985058'},
    'degree_type': 'other',
    'curated_relation': False},
   {'name': 'Ponce, William A.',
    'hidden': False,
    'degree_type': 'master',
    'curated_relation': False}],
  'email_addresses': [{'value': 'restrepo@udea.edu.co', 'current': True}],
  'positions': [{'rank': 'SENIOR',
    'hidden': False,
    'record': {'$ref': 'https://inspirehep.net/api/institutions/903906'},
    'current': True,
    'start_date': '2004',
    'institution': 'Antioquia U.',
    'curated_relation': True},
   {'rank': 'VISITOR',
    'hidden': False,
    'record': {'$ref': 'https://inspirehep.net/api/institutions/902714'},
    'current': False,
    'end_date': '2022',
    'start_date': '2022',
    'institution': 'Campinas S